In [1]:
import os
import numpy as np
import pandas as pd
from tifffile import imread
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [2]:
base_path = '/home/unix/jiahao/wanglab/Data/Analyzed/2024-02-06-Hongyu-Hongyu_multicap_covid_spleen_replicates1/'
dapi_path = os.path.join(base_path, 'images/flamingo/DAPI_MAX')
signal_path = os.path.join(base_path, 'signal')
output_path = os.path.join(base_path, "output")
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [16]:
current_sample = 'sample3'

current_dapi_img = imread(os.path.join(base_path, "images/fused/2D", current_sample, "DAPI.tif"))
current_signal = pd.read_csv(os.path.join(signal_path, current_sample + '_goodSpots.csv'))
current_signal['x'] = current_signal['x'] - 1
current_signal['y'] = current_signal['y'] - 1
current_signal['z'] = current_signal['z'] - 1

fig_size = np.array([current_dapi_img.shape[1], current_dapi_img.shape[0]]) / 10000 * 4

all_genes = sorted(current_signal.gene.unique())
for current_gene in tqdm(all_genes):
    # current_gene = 'Cd68'
    current_spots = current_signal[current_signal['gene'] == current_gene]
    n_spots = current_spots.shape[0]
    fig, ax = plt.subplots(figsize=fig_size)
    ax.imshow(current_dapi_img, cmap='gray')
    # sns.scatterplot(data=current_spots, x='x', y='y', color='red', size=1, linewidth=0)
    ax.plot(current_spots['x'], current_spots['y'], 'r.', markersize=1)
    ax.set_facecolor('k')
    plt.axis('off')
    plt.tight_layout()
    # plt.savefig(os.path.join(output_path, f'{current_sample}_{current_gene}_{n_spots}.tif'), pil_kwargs={"compression": "tiff_lzw"})
    plt.savefig(os.path.join(output_path, f'{current_sample}_{current_gene}_{n_spots}.png'))
    plt.clf()
    plt.close()
    # plt.show()

  0%|          | 0/61 [00:00<?, ?it/s]

In [33]:
import numpy as np
from tifffile import imread, imwrite
from skimage.util import img_as_ubyte, img_as_float, invert
from skimage.exposure import rescale_intensity
from skimage.morphology import disk
from skimage.filters import median

In [4]:
current_fov_id = "Position001"

# define the path to the images
dapi_path = os.path.join(base_path, 'images/flamingo/DAPI')
flamingo_path = os.path.join(base_path, 'images/flamingo/Flamingo')

# load the images
current_dapi_img = imread(os.path.join(dapi_path, current_fov_id + '.tif'))
current_flamingo_img = imread(os.path.join(flamingo_path, current_fov_id + '.tif'))

# enhance the contrast of the images (flamingo)
eh_value = 0.005
vmin = np.quantile(current_flamingo_img, eh_value)
vmax = np.quantile(current_flamingo_img, 1-eh_value)
vrange = (vmin, vmax)
current_flamingo_img_eh = rescale_intensity(current_flamingo_img, vrange)

# enhance the contrast of the images (dapi)
eh_value = 0.001
vmin = np.quantile(current_dapi_img, eh_value)
vmax = np.quantile(current_dapi_img, 1-eh_value)
vrange = (vmin, vmax)
current_dapi_img_eh = rescale_intensity(current_dapi_img, vrange)


In [34]:
# create the output image
current_dapi_img_eh = img_as_float(current_dapi_img_eh)
current_flamingo_img_eh = img_as_float(current_flamingo_img_eh)
current_flamingo_invert = invert(current_flamingo_img_eh)

current_output_img = current_dapi_img_eh * current_flamingo_invert
current_output_img = img_as_ubyte(current_output_img)

imwrite(os.path.join(output_path, f'{current_fov_id}.tif'), current_output_img)